In [1]:
import mxnet as mx
from mxnet import init, gluon, nd, autograd, image
from mxnet.gluon import nn
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os
from glob import glob
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ctx = mx.gpu()

In [2]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [3]:
n = len(glob('Images/*/*.jpg'))

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

net = models.get_model('inceptionv3', pretrained=True, ctx=ctx)
features = []
for j in range(0,161):
    print(j, end='-')
    i = 0
    temp = nd.zeros((128, 3, 299, 299))
    for file_name in glob('Images/*/*.jpg')[128*j:128*(j+1)]:
        img = cv2.imread(file_name)
        img_299 = ((cv2.resize(img, (299, 299))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        temp[i] = nd.array(img_299)
        nd.waitall()
        i = i + 1
    if j == 160:
        temp = temp[0:100]
    data_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(temp), batch_size=128)
    for data in data_iter_299:
        feature = net.features(data.as_in_context(mx.gpu()))
        feature = gluon.nn.Flatten()(feature)
        features.append(feature.as_in_context(mx.cpu()))
    nd.waitall()
features = nd.concat(*features, dim=0)
print(features.shape)
nd.save('features_incep.nd', features)

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,(20580, 2048)


In [ ]:
n = len(glob('Images/*/*.jpg'))

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

net = models.get_model('resnet152_v1', pretrained=True, ctx=ctx)
features = []
for j in range(0,161):
    print(j,end='-')
    i = 0
    temp = nd.zeros((128, 3, 224, 224))
    for file_name in glob('Images/*/*.jpg')[128*j:128*(j+1)]:
        img = cv2.imread(file_name)
        img_224 = ((cv2.resize(img, (224, 224))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
        temp[i] = nd.array(img_224)
        nd.waitall()
        i = i + 1
    if j == 160:
        temp = temp[0:100]
    data_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(temp), batch_size=128)
    for data in data_iter_224:
        feature = net.features(data.as_in_context(mx.gpu()))
        feature = gluon.nn.Flatten()(feature)
        features.append(feature.as_in_context(mx.cpu()))
    nd.waitall()
features = nd.concat(*features, dim=0)
print(features.shape)
nd.save('features_res.nd', features)